In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import faculty.datasets as datasets
from pandas.api.types import CategoricalDtype
import matplotlib.ticker as mtick
import statsmodels.api as sm
import plotly.graph_objects as go


from pandas import Series, DataFrame
import os
from functools import reduce
vcvtype = 'nonrobust'


In [3]:
all_football_18 = pd.read_csv('/project/all_football_18.csv', index_col=0)

In [4]:
all_football_18.head()

,country_name,Final_positional_ranking,goal,shotsPerGame,yellow_card,red_card,possession_percentage,passSuccess_percentage,shotOnTargetPG,dribbleWonPG,...,keyPassShort,left_side_percent,middle_side_percent,right_side_percent,own_third_percent,middle_third_percent,opposition_third_percent,shot_left_side_percent,shot_middle_side_percent,shot_right_side_percent
0,Argentina,16,6,13.8,11,0,61.1,85.0,4.5,13.5,...,8.8,38,27,35,19,49,32,27,49,24
1,Australia,25,2,11.0,7,0,50.9,83.7,2.3,11.0,...,6.3,35,20,45,27,47,25,9,79,12
2,Belgium,3,16,15.1,11,0,52.1,86.3,5.7,13.3,...,10.6,34,26,40,27,47,26,26,56,18
3,Brazil,5,8,20.8,7,0,56.8,87.9,8.0,14.8,...,13.0,43,29,29,25,44,31,28,60,13
4,Colombia,10,6,9.8,9,1,50.2,79.9,3.3,7.8,...,5.0,33,22,45,29,48,23,7,66,27


In [5]:
#rename position columns to have the "percent"
all_football_18 = all_football_18.rename(columns={"left_side": "left_side_percent",
                                                 "middle_side":"middle_side_percent",
                                                 "right_side":"right_side_percent",
                                                 "own_third":"own_third_percent",
                                                 "middle_third":"middle_third_percent",
                                                 "opposition_third":"opposition_third_percent",
                                                 "shot_left_side":"shot_left_side_percent",
                                                 "shot_middle_side":"shot_middle_side_percent",
                                                 "shot_right_side":"shot_right_side_percent"})

all_football_18 = all_football_18.replace({'%': ''}, regex=True)
all_football_18.head()

,country_name,Final_positional_ranking,goal,shotsPerGame,yellow_card,red_card,possession_percentage,passSuccess_percentage,shotOnTargetPG,dribbleWonPG,...,keyPassShort,left_side_percent,middle_side_percent,right_side_percent,own_third_percent,middle_third_percent,opposition_third_percent,shot_left_side_percent,shot_middle_side_percent,shot_right_side_percent
0,Argentina,16,6,13.8,11,0,61.1,85.0,4.5,13.5,...,8.8,38,27,35,19,49,32,27,49,24
1,Australia,25,2,11.0,7,0,50.9,83.7,2.3,11.0,...,6.3,35,20,45,27,47,25,9,79,12
2,Belgium,3,16,15.1,11,0,52.1,86.3,5.7,13.3,...,10.6,34,26,40,27,47,26,26,56,18
3,Brazil,5,8,20.8,7,0,56.8,87.9,8.0,14.8,...,13.0,43,29,29,25,44,31,28,60,13
4,Colombia,10,6,9.8,9,1,50.2,79.9,3.3,7.8,...,5.0,33,22,45,29,48,23,7,66,27


Export all_football_18 with the new updates

In [6]:
all_football_18.to_csv("all_football_18.csv")

In [13]:
# Extracting the column names as a list in order to use them for the regression
list(all_football_18.columns)

['country_name',
 'Final_positional_ranking',
 'goal',
 'shotsPerGame',
 'yellow_card',
 'red_card',
 'possession_percentage',
 'passSuccess_percentage',
 'shotOnTargetPG',
 'dribbleWonPG',
 'foulGivenPG',
 'shotsConcededPG',
 'tacklePG',
 'interceptionPG',
 'foulsPG',
 'offsideGivenPG',
 'clearanceTotal',
 'duelAerialTotal',
 'duelAerialWon',
 'duelAerialLost',
 'stat-value',
 'assistCross',
 'assistCorner',
 'assistThroughball',
 'assistFreekick',
 'assistOther',
 'assist',
 'passLongBallAccurate',
 'passLongBallInaccurate',
 'shortPassAccurate',
 'shortPassInaccurate',
 'keyPassLong',
 'keyPassShort',
 'left_side_percent',
 'middle_side_percent',
 'right_side_percent',
 'own_third_percent',
 'middle_third_percent',
 'opposition_third_percent',
 'shot_left_side_percent',
 'shot_middle_side_percent',
 'shot_right_side_percent']

We ran a regression with all the football variables as independent variables (X) and chose our dependent variable, Y, to be the "Final_positional_ranking" in order to see which variables significantly affected the final position of the teams.

In [12]:
# All variables
X = all_football_18[['red_card','goal','possession_percentage',
                     'passSuccess_percentage',
 'tacklePG',
 'stat-value',
 'assistCross',
 'assistCorner',
 'assistThroughball',
 'assistFreekick',
 'assistOther',
 'assist',
 'passLongBallAccurate',
 'passLongBallInaccurate',
 'shortPassAccurate',
 'shortPassInaccurate',
 'keyPassLong',
 'keyPassShort',
 'left_side_percent',
 'middle_side_percent',
 'right_side_percent',
 'own_third_percent',
 'middle_third_percent',
 'opposition_third_percent',
 'shot_left_side_percent',
 'shot_middle_side_percent',
 'shot_right_side_percent']]
X = sm.add_constant(X)
Y = all_football_18['Final_positional_ranking']

# Regression 
model = sm.OLS(Y, X.astype(float))
results = model.fit(cov_type=vcvtype)
print(results.summary())

                               OLS Regression Results                               
Dep. Variable:     Final_positional_ranking   R-squared:                       0.981
Model:                                  OLS   Adj. R-squared:                  0.854
Method:                       Least Squares   F-statistic:                     7.741
Date:                      Mon, 20 Dec 2021   Prob (F-statistic):             0.0295
Time:                              16:58:04   Log-Likelihood:                -52.934
No. Observations:                        32   AIC:                             161.9
Df Residuals:                             4   BIC:                             202.9
Df Model:                                27                                         
Covariance Type:                  nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

We can see that from our regression above, there aren't any very significant variables affecting the final ranking position. We therefore, decided to remove the variables regarding the positions which were in percentages.

In [11]:
# Percentage variables
X = all_football_18[['red_card','goal','possession_percentage',
                     'passSuccess_percentage',
 'tacklePG',
 'stat-value',
 'assistCross',
 'assistCorner',
 'assistThroughball',
 'assistFreekick',
 'assistOther',
 'assist',
 'passLongBallAccurate',
 'passLongBallInaccurate',
 'shortPassAccurate',
 'shortPassInaccurate',
 'keyPassLong',
 'keyPassShort']]
X = sm.add_constant(X)
Y = all_football_18['Final_positional_ranking']

# Regression 
model = sm.OLS(Y, X)
results = model.fit(cov_type=vcvtype)
print(results.summary())

                               OLS Regression Results                               
Dep. Variable:     Final_positional_ranking   R-squared:                       0.935
Model:                                  OLS   Adj. R-squared:                  0.846
Method:                       Least Squares   F-statistic:                     10.44
Date:                      Mon, 20 Dec 2021   Prob (F-statistic):           5.24e-05
Time:                              16:57:59   Log-Likelihood:                -72.735
No. Observations:                        32   AIC:                             183.5
Df Residuals:                            13   BIC:                             211.3
Df Model:                                18                                         
Covariance Type:                  nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

From our regression, we can deduct that 'goal', 'possession_percentage', 'stat-value', 'passLongBallInaccurate', 'shortPassAccurate' and 'keyPassShort' are the variables which mainly affected the final position ranking of the teams as their p-value was close to 0.

In [10]:
# Significant variables
X = all_football_18[['goal','possession_percentage',
 'stat-value',
 'passLongBallInaccurate',
 'shortPassAccurate',
 'keyPassShort']]
X = sm.add_constant(X)
Y = all_football_18['Final_positional_ranking']

# Regression 
model = sm.OLS(Y, X)
results = model.fit(cov_type=vcvtype)
print(results.summary())

                               OLS Regression Results                               
Dep. Variable:     Final_positional_ranking   R-squared:                       0.887
Model:                                  OLS   Adj. R-squared:                  0.860
Method:                       Least Squares   F-statistic:                     32.72
Date:                      Mon, 20 Dec 2021   Prob (F-statistic):           1.14e-10
Time:                              16:57:53   Log-Likelihood:                -81.645
No. Observations:                        32   AIC:                             177.3
Df Residuals:                            25   BIC:                             187.6
Df Model:                                 6                                         
Covariance Type:                  nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------